In [1]:
import pandas as pd
import numpy as np

In [2]:
paneldata = pd.read_csv("panelregressioninputs.csv")
paneldata_eduqua = pd.read_csv("panelregressioninputs_eduqua.csv")

In [3]:
year  = pd.Categorical(paneldata['year'])
paneldata = paneldata.set_index(["country", "year"])
paneldata['year'] = year

year  = pd.Categorical(paneldata_eduqua['year'])
paneldata_eduqua = paneldata_eduqua.set_index(["country", "year"])
paneldata_eduqua['year'] = year

In [4]:
paneldata.columns

Index(['gdpgrowth', 'tfpgrowth', 'gdp', 'tfp', 'sexratio', 'maleyoung',
       'femaleyoung', 'totalyoung', 'logfertilityrate', 'loglifeexpectancy',
       'gcratio', 'inflation', 'ruleoflaw', 'vanda', 'vandasquared',
       'femaleschooling', 'year'],
      dtype='object')

In [5]:
paneldata.head(6)

gdpgrowth  tfpgrowth        gdp       tfp  sexratio  \
country   year                                                        
Argentina 2000   0.104004   0.010692  13.175429 -0.139970  1.077512   
          2005   0.320711   0.105593  13.274373 -0.129335  1.080142   
          2010   0.129353  -0.014859  13.552543 -0.028953  1.081039   
Armenia   2000   0.808074   0.900307   9.304577 -0.691459  1.211430   
          2005   0.208409   0.006927   9.896839 -0.049444  1.244295   
          2010   0.200263   0.144122  10.086143 -0.042541  1.303781   

                maleyoung  femaleyoung  totalyoung  logfertilityrate  \
country   year                                                         
Argentina 2000  29.052199     0.269623   27.984825          0.922909   
          2005  27.979347     0.259034   26.918893          0.881699   
          2010  26.904121     0.248873   25.873779          0.855479   
Armenia   2000  28.445319     0.234808   25.811119          0.498713   
          2005  23.952968     0.192502   21.461537          0.528980   
          2010  22.214877     0.170388   19.460740          0.513572   

                loglifeexpectancy   gcratio  inflation  ruleoflaw     vanda  \
country   year                                                                
Argentina 2000           4.306041  0.125457   0.083449  -0.391472  0.184178   
          2005           4.317588  0.134196   0.088477  -0.776114  0.007864   
          2010           4.327228  0.160761   0.102239  -0.829332  0.177265   
Armenia   2000           4.278934  0.106924   0.030196  -0.718949 -0.650817   
          2005           4.300386  0.108761   0.040598  -0.623014 -0.910273   
          2010           4.309398  0.122109   0.060455  -0.728167 -1.094504   

                vandasquared  femaleschooling  year  
country   year                                       
Argentina 2000      0.033922             8.62  2000  
          2005      0.000062             9.21  2005  
          2010      0.031423             9.48  2010  
Armenia   2000      0.423563            10.69  2000  
          2005      0.828596            10.75  2005  
          2010      1.197939            10.84  2010

In [6]:
from linearmodels.panel import PooledOLS
import statsmodels.api as sm

#pooled regressions gdp growth
exog_vars = ['gdp', 'femaleschooling', 
       'logfertilityrate', 'loglifeexpectancy', 'gcratio', 'inflation',
       'ruleoflaw', 'vanda', 'vandasquared', 'year']
exog = sm.add_constant(paneldata[exog_vars])

mod = PooledOLS(paneldata['gdpgrowth'], exog)
fe_res_gdp = mod.fit(cov_type = 'robust')

#pooled regressions tfp growth

mod = PooledOLS(paneldata['tfpgrowth'], exog)
fe_res_tfp = mod.fit(cov_type = 'robust')

C:\Users\fjying\anaconda3\lib\site-packages\linearmodels\shared\exceptions.py:35: MissingValueWarning: 
Inputs contain missing values. Dropping rows with missing observations.
  warnings.warn(missing_value_warning_msg, MissingValueWarning)


from linearmodels.panel import PanelOLS
#fe model controlling for year
#add year dummies
exog_vars = ['gdp', 'femaleschooling', 
       'logfertilityrate', 'loglifeexpectancy', 'gcratio', 'inflation',
       'ruleoflaw', 'vanda', 'vandasquared']
exog = sm.add_constant(paneldata[exog_vars])
exog = sm.add_constant(paneldata[exog_vars])
mod = PanelOLS(paneldata.gdpgrowth, exog, time_effects = True)
fe_res_gdp = mod.fit()

exog_vars = [ 'gdp', 'femaleschooling', 
       'logfertilityrate', 'loglifeexpectancy', 'gcratio', 'inflation',
       'ruleoflaw', 'vanda', 'vandasquared']
exog = sm.add_constant(paneldata[exog_vars])
exog = sm.add_constant(paneldata[exog_vars])
mod = PanelOLS(paneldata.tfpgrowth, exog, time_effects = True)
fe_res_tfp = mod.fit()

print(fe_res_gdp)
print(fe_res_tfp)


In [7]:
from linearmodels.panel import compare
print(compare({"GDP Growth": fe_res_gdp, "TFP Growth": fe_res_tfp}, precision = 'std_errors', stars = True))
#add stars to indicate statistical significance

                Model Comparison               
                        GDP Growth   TFP Growth
-----------------------------------------------
Dep. Variable            gdpgrowth    tfpgrowth
Estimator                PooledOLS    PooledOLS
No. Observations               274          274
Cov. Est.                   Robust       Robust
R-squared                   0.2704       0.2544
R-Squared (Within)          0.1280       0.2365
R-Squared (Between)         0.3524       0.2617
R-Squared (Overall)         0.2704       0.2544
F-statistic                 8.8264       8.1255
P-value (F-stat)            0.0000       0.0000
===================== ============ ============
const                       0.3612     0.9859**
                          (0.6302)     (0.4858)
gdp                        -0.0027     -0.0091*
                          (0.0053)     (0.0047)
femaleschooling             0.0034    0.0118***
                          (0.0046)     (0.0041)
logfertilityrate            0.0319    -0

In [8]:
#Instrumental Variable Estimation
from linearmodels.iv.model import IV2SLS

exog_vars = ['gdp',
       'logfertilityrate', 'loglifeexpectancy', 'gcratio', 'inflation',
       'ruleoflaw', 'vanda', 'vandasquared', 'year']
exog = sm.add_constant(paneldata[exog_vars])

iv_gdp = IV2SLS(dependent = paneldata['gdpgrowth'],
            exog= exog,
            endog= paneldata['femaleschooling'],
            instruments= paneldata['femaleyoung']).fit(cov_type = 'robust')

iv_tfp = IV2SLS(dependent = paneldata['tfpgrowth'],
            exog= exog,
            endog= paneldata['femaleschooling'],
            instruments= paneldata['femaleyoung']).fit(cov_type = 'robust')

In [9]:
exog_vars = ['gdp', 'femaleyoung', 
       'logfertilityrate', 'loglifeexpectancy', 'gcratio', 'inflation',
       'ruleoflaw', 'vanda', 'vandasquared', 'year']
exog = sm.add_constant(paneldata[exog_vars])

mod = PooledOLS(paneldata['femaleschooling'], exog)
first_stage_1 = mod.fit(cov_type = 'robust')

In [10]:
from linearmodels.iv.results import compare
print(compare({"GDP Growth IV": iv_gdp, 'TFP Growth IV': iv_tfp}, precision = 'std_errors', stars = True))

                 Model Comparison                 
                      GDP Growth IV  TFP Growth IV
--------------------------------------------------
Dep. Variable             gdpgrowth      tfpgrowth
Estimator                   IV-2SLS        IV-2SLS
No. Observations                274            274
Cov. Est.                    robust         robust
R-squared                    0.0856         0.2005
Adj. R-squared               0.0472         0.1669
F-statistic                  108.22         54.528
P-value (F-stat)             0.0000      9.453e-08
===================     ===========   ============
const                        0.1312       1.0774**
                           (0.7076)       (0.5064)
gdp                         -0.0066        -0.0076
                           (0.0058)       (0.0048)
logfertilityrate            -0.1383        -0.0088
                           (0.1110)       (0.0771)
loglifeexpectancy            0.1456       -0.2527*
                           (0.1

# Add eduquality

In [11]:
paneldata_eduqua.columns

Index(['gdpgrowth', 'tfpgrowth', 'gdp', 'tfp', 'sexratio', 'maleyoung',
       'femaleyoung', 'totalyoung', 'logfertilityrate', 'loglifeexpectancy',
       'gcratio', 'inflation', 'ruleoflaw', 'vanda', 'vandasquared',
       'eduquality', 'femaleschooling', 'year'],
      dtype='object')

In [12]:
from linearmodels.panel import PooledOLS
import statsmodels.api as sm

#pooled regressions
exog_vars = ['gdp', 'femaleschooling', 'eduquality',
       'logfertilityrate', 'loglifeexpectancy', 'gcratio', 'inflation',
       'ruleoflaw', 'vanda', 'vandasquared', 'year']
exog = sm.add_constant(paneldata_eduqua[exog_vars])

mod = PooledOLS(paneldata_eduqua['gdpgrowth'], exog)
fe_res_gdp = mod.fit(cov_type = 'robust')

#pooled regressions
mod = PooledOLS(paneldata_eduqua['tfpgrowth'], exog)
fe_res_tfp = mod.fit(cov_type = 'robust')

In [13]:
from linearmodels.panel import compare
print(compare({"GDP Growth": fe_res_gdp, "TFP Growth": fe_res_tfp}, precision = 'std_errors', stars = True))
#add stars to indicate statistical significance
#insignifcant of years of schooling may also occurs because of no new period data from 2010 to 2014 is used

                Model Comparison               
                        GDP Growth   TFP Growth
-----------------------------------------------
Dep. Variable            gdpgrowth    tfpgrowth
Estimator                PooledOLS    PooledOLS
No. Observations                96           96
Cov. Est.                   Robust       Robust
R-squared                   0.5560       0.4803
R-Squared (Within)          0.2457       0.3966
R-Squared (Between)         0.7939       0.5512
R-Squared (Overall)         0.5560       0.4803
F-statistic                 9.5641       7.0567
P-value (F-stat)            0.0000       0.0000
===================== ============ ============
const                       1.1833      -0.1106
                          (0.9040)     (0.8086)
gdp                     -0.0274***    -0.0256**
                          (0.0085)     (0.0112)
femaleschooling             0.0064       0.0079
                          (0.0074)     (0.0097)
eduquality                  0.0163    0.

In [14]:
from linearmodels.iv.model import IV2SLS

exog_vars = ['gdp', 'eduquality',
       'logfertilityrate', 'loglifeexpectancy', 'gcratio', 'inflation',
       'ruleoflaw', 'vanda', 'vandasquared', 'year']
exog = sm.add_constant(paneldata_eduqua[exog_vars])

iv_gdp = IV2SLS(dependent = paneldata_eduqua['gdpgrowth'],
            exog= exog,
            endog= paneldata_eduqua['femaleschooling'],
            instruments= paneldata_eduqua['femaleyoung']).fit(cov_type = 'robust')

iv_tfp = IV2SLS(dependent = paneldata_eduqua['tfpgrowth'],
            exog= exog,
            endog= paneldata_eduqua['femaleschooling'],
            instruments= paneldata_eduqua['femaleyoung']).fit(cov_type = 'robust')

In [15]:
exog_vars = ['gdp', 'femaleyoung', 'eduquality',
       'logfertilityrate', 'loglifeexpectancy', 'gcratio', 'inflation',
       'ruleoflaw', 'vanda', 'vandasquared', 'year']

exog = sm.add_constant(paneldata_eduqua[exog_vars])

first_stage_2 = PooledOLS(paneldata_eduqua['femaleschooling'], exog)
first_stage_2 = first_stage_2.fit(cov_type = 'robust')

In [16]:
from linearmodels.panel import compare
print(compare({"First Stage 1": first_stage_1, "First Stage 2": first_stage_2}, precision = 'std_errors', stars = True))

                     Model Comparison                    
                          First Stage 1     First Stage 2
---------------------------------------------------------
Dep. Variable           femaleschooling   femaleschooling
Estimator                     PooledOLS         PooledOLS
No. Observations                    274                96
Cov. Est.                        Robust            Robust
R-squared                        0.7102            0.6515
R-Squared (Within)               0.5479           -0.0150
R-Squared (Between)              0.7014            0.6744
R-Squared (Overall)              0.7102            0.6515
F-statistic                      58.375            14.276
P-value (F-stat)                 0.0000            0.0000
===================== ================= =================
const                           -4.7127            18.022
                               (5.2721)          (13.112)
gdp                            -0.1132*         -0.2236**
              

print(iv_gdp.wu_hausman())  
print(iv_tfp.wu_hausman())  

In [17]:
from linearmodels.iv.results import compare
print(compare({"GDP Growth IV": iv_gdp, 'TFP Growth IV': iv_tfp}, precision = 'std_errors', stars = True))

                 Model Comparison                
                      GDP Growth IV TFP Growth IV
-------------------------------------------------
Dep. Variable             gdpgrowth     tfpgrowth
Estimator                   IV-2SLS       IV-2SLS
No. Observations                 96            96
Cov. Est.                    robust        robust
R-squared                   -26.224       -7.6432
Adj. R-squared              -29.789       -8.7750
F-statistic                  2.2285        5.9064
P-value (F-stat)             0.9975        0.8795
===================     ===========   ===========
const                        9.8750        4.4324
                           (19.065)      (9.4624)
gdp                         -0.1420       -0.0856
                           (0.2511)      (0.1267)
eduquality                   0.8416        0.4815
                           (1.7761)      (0.8813)
logfertilityrate             0.0757       -0.0103
                           (0.3650)      (0.1886)


In [19]:
#investigate further
df = pd.read_csv("panelregressioninputs_eduqua.csv")

In [22]:
df['country'].drop_duplicates()

0              Argentina
2                Armenia
4              Australia
6                Austria
8                Bahrain
10               Belgium
12              Botswana
14                Brazil
16              Bulgaria
18                Canada
20                 Chile
22                Cyprus
24        Czech Republic
26               Denmark
28               Estonia
30               Finland
32                France
34               Germany
36                Greece
38               Hungary
40               Iceland
42             Indonesia
44               Ireland
46                 Italy
48                 Japan
50                Jordan
52                Latvia
54             Lithuania
56              Malaysia
58                Mexico
60               Morocco
62           Netherlands
64           New Zealand
66                Norway
68                  Peru
70                Poland
72              Portugal
74               Romania
76    Russian Federation
78          Saudi Arabia
